In [ ]:
# 카메라(/dev/video0) 데이터의 1프레임을 YOLO로 인식한 후 객체 수 데이터를 서버로 전송하는 코드 (클라이언트)

import cv2
from ultralytics import YOLO

model = YOLO("best.pt")

cap = cv2.VideoCapture(0)
results = None

# 1프레임 실행 이후 수동 종료 필요
while cap.isOpened():
  # Read a frame from the video
  success, frame = cap.read()
  
  if success:
    # run model on the frame, persisting tracks between frames
    results = model.track(frame, persist = True)
    
    annotated_frame = results[0].plot()

    if len(results) == False:
      print(results[0].names)

    # cv2.imshow("FIRE TEST", results[-1])

/home/park/dev_ws/media_pipe/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


0: 480x640 1 smoke, 127.3ms
Speed: 6.7ms preprocess, 127.3ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 smoke, 80.9ms
Speed: 1.5ms preprocess, 80.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 smokes, 95.1ms
Speed: 1.0ms preprocess, 95.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 smokes, 95.6ms
Speed: 2.0ms preprocess, 95.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 smokes, 89.4ms
Speed: 2.0ms preprocess, 89.4ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 smokes, 89.8ms
Speed: 2.3ms preprocess, 89.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 smoke, 91.3ms
Speed: 1.9ms preprocess, 91.3ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 smokes, 82.4ms
Speed: 2.2ms preprocess, 82.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480,

KeyboardInterrupt: 

In [2]:
# 2. 카메라 수동 종료후 꺼주기 작업 필요
cap.release()

In [3]:
# 3. 1프레임 정보 확인
results[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'smoke', 1: 'fire'}
obb: None
orig_img: array([[[115, 135, 134],
        [116, 137, 135],
        [118, 135, 137],
        ...,
        [113, 129, 143],
        [117, 130, 144],
        [115, 129, 143]],

       [[117, 135, 134],
        [118, 136, 135],
        [118, 136, 135],
        ...,
        [113, 129, 143],
        [115, 131, 144],
        [115, 131, 144]],

       [[119, 132, 135],
        [121, 133, 136],
        [117, 134, 136],
        ...,
        [115, 131, 144],
        [116, 132, 145],
        [117, 133, 146]],

       ...,

       [[ 31,  31,  33],
        [ 22,  21,  24],
        [ 23,  22,  25],
        ...,
        [ 29,  29,  29],
        [ 27,  27,  27],
        [ 29,  29,  29]],

       [[ 26,  25,  27],
        [ 22,  21,  24],
        [ 26,  25,  27],
        ...,
        [ 30,  30,  30],
        [ 28,  28,  28],
   

In [4]:
# 4. 전송할 데이터 구조 생성
import time
import json

def generateData():
  data = {
    "timestamp" : time.time(),
    "detections" : results[0].names,
  }
  return data

data = generateData()

# 파이썬 딕셔너리를 json 문자열로 <str> 변환 (직렬화)
json_string = json.dumps(data, indent=2, ensure_ascii=False)

# 파이썬 딕셔너리를 json 문자열로 <str> 변환 (직렬화)
data = json.dumps(data, ensure_ascii=False)


# str 문자열화된 json 데이터를 utf-8 바이트 데이터로 변환 (인코딩)
message_bytes = data.encode('utf-8')

In [17]:
# 5. 전송 함수 코드 작성
import socket

HOST = 'localhost' # 로컬호스트
PORT = 6600 # 6600번 포트 사용

def send(data):
  with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    print(f"시스템 : 연결 대기 - {HOST} : {PORT}")
    s.connect((HOST, PORT))
    # s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # -> 소켓을 여는 측은 클라이언트가 아니라 서버이어야 함
    # s.bind((HOST, PORT))
    # s.listen()
    print(f"시스템 : 연결 완료 - {HOST} : {PORT}")
    
    # conn, addr = s.accept() # -> 연결을 수락하는측은 서버이고, 클라이언트는 accept하지 않는다.
    
    # with s:
    # 연결시 실행 동작
    try:
      while True:
        # 바이트 데이터 전송
        s.sendall(data)
        print("\n전송 완료")
        
        # 1초 대기
        time.sleep(1)
    except (BrokenPipeError, ConnectionResetError):
      print(f"연결 끊어짐")
    finally:
      print("서버 종료")

In [18]:
# 6. 생성한 인코딩 데이터 확인
print(message_bytes)
print(type(message_bytes))

b'{"timestamp": 1756969480.6353636, "detections": {"0": "smoke", "1": "fire"}}'
<class 'bytes'>


In [21]:
# 7. 전송 함수를 이용하여 변환된 인코딩 데이터 전송
# -> 서버 코드 실행시 전송 작업 수행함 (test_server.py)
send(message_bytes)

시스템 : 연결 대기 - localhost : 6600
시스템 : 연결 완료 - localhost : 6600

전송 완료

전송 완료

전송 완료

전송 완료

전송 완료

전송 완료

전송 완료
서버 종료


KeyboardInterrupt: 